In [2]:
# Audio Signals Classification - This notebook loads a trained model and classifies all WAV files in a specified folder.

# Import necessary libraries
import os
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Load the trained model
model = tf.keras.models.load_model('emotion_recognition_model.h5')

# Define feature extraction function
def extract_features(data, sample_rate):
    result = np.array([])
    
    # Zero Crossing Rate
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result = np.hstack((result, zcr))
    
    # Chroma STFT
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft))
    
    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc))
    
    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms))
    
    # Mel Spectrogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel))
    
    return result

# Function to classify audio files in a folder
def classify_audio_files(folder_path):
    file_list = os.listdir(folder_path)
    X = []
    file_paths = []

    for file in file_list:
        if file.endswith('.wav'):
            file_path = os.path.join(folder_path, file)
            data, sample_rate = librosa.load(file_path, duration=2.5, offset=0.6)
            features = extract_features(data, sample_rate)
            X.append(features)
            file_paths.append(file_path)
    
    # Standardize the data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = np.expand_dims(X, axis=2)

    # Predict the emotions
    predictions = model.predict(X)
    predicted_labels = np.argmax(predictions, axis=1)
    
    # Decode the predicted labels
    emotion_dict = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
    predicted_emotions = [emotion_dict[label] for label in predicted_labels]

    # Create a DataFrame to display the results
    results_df = pd.DataFrame({'File Path': file_paths, 'Predicted Emotion': predicted_emotions})
    return results_df

# Specify the folder containing the WAV files
folder_path = 'jel_recordings'

# Classify the audio files in the folder
results_df = classify_audio_files(folder_path)

# Display the results
display(results_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


,File Path,Predicted Emotion
0,jel_recordings/02.wav,sad
1,jel_recordings/16.wav,angry
2,jel_recordings/17.wav,sad
3,jel_recordings/03.wav,disgust
4,jel_recordings/15.wav,angry
5,jel_recordings/01.wav,disgust
6,jel_recordings/14.wav,fear
7,jel_recordings/10.wav,angry
8,jel_recordings/04.wav,disgust
9,jel_recordings/05.wav,angry
